In [ ]:
import pandas as pd
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.spatial import cKDTree
from scipy.ndimage import gaussian_filter
from skimage import graph
from skimage.graph import MCP
from io import StringIO

import pygmt as gmt
import xarray as xr

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs

from scripts import morphoGrid as morph
from scipy.spatial import cKDTree

import meshio

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Build a VTK mesh from Badlands output

### Intermittent flooding

+ -415 (584) -400 (600) -393 (607)
+ -337 (663) -325 (675) -304 (696) 
+ -246 (754) -237 (763) -230 (770)
+ -220 (780) -213 (787) -190 (810)
+ -107 (893) -81 (919)  -70 (930)

### Full flooding

+ -132 (868) -124 (876) -112 (888)
+ -16 (984) 0 (1000)


steps to use:

+ 430 
+ 600 
+ 675 
+ 763 
+ 787 
+ 919 


+ 625
+ 930

In [ ]:
step = 625
dx = 5000.
folder = 'lem-outputs/lowsub/h5'

badland_topo = morph.morphoGrid(folder=folder, bbox = None, dx=dx)
badland_topo.loadHDF5(timestep=step)
height = badland_topo.z.shape[0]
width = badland_topo.z.shape[1]
minx = badland_topo.x.min()
maxy = badland_topo.y.max()
sealevel = badland_topo.sl
cells = badland_topo.cells-1
area = badland_topo.area
vertices = badland_topo.vertices
fa = badland_topo.fa
ed = badland_topo.ed

vertices[:,-1] -= sealevel

In [ ]:
vtkfile = 'vtk/map'+str(step)+'.vtk'
vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, 
                                   point_data={"Z":vertices[:,-1], "ED":ed, 
                                               'FA':fa
                                              })
meshio.write(vtkfile, vis_mesh)

# Compute LEC

In [ ]:
#!cd scripts; mpirun -n 8  python3 -m mpi4py.futures  mpi_sunda.py -i $vtkfile

# Build a regular cost grid

In [ ]:
# Regular grid coordinates
xyi = np.dstack([badland_topo.x.flatten(), badland_topo.y.flatten()])[0]
# TIN grid coordinates
XY = vertices[:,:2] 
tin_tree = cKDTree(XY)


# Get costs 
costmesh = 'vtk/'+'map'+str(step)+'.cost.vtk'
mesh = meshio.read(costmesh)

# vertices = mesh.points
# cells = mesh.cells_dict['triangle']
elev = mesh.point_data['Z']
cost = mesh.point_data['cost']
cost = 1.0-cost/cost.max()
ED = mesh.point_data['ED']
FA = mesh.point_data['FA']

# set a limit on flow accumulation to only consider large rivers and 
# use kdtree to find closest points to these rivers on the mesh
logFA = np.log10(FA) / 13. # 13 is coming from logFA.max() at step 430 when river discharge is maximum
riverIDs = np.where(logFA>0.8)[0]
rtin_tree = cKDTree(XY[riverIDs,:]) 
dist, ids = rtin_tree.query(XY, k=1)
dist[elev<0] = 1.e18 # marine points set to large distance

In [ ]:
# Define categorical distances cost based on user requirement
landIDs = np.where(elev>=0)[0]
cat_dist = np.zeros(len(dist))

cat_dist[elev<0] = 20.
cat_dist[landIDs] = 20.
cat_dist[dist<50000] = 14.
cat_dist[dist<35000] = 11.
cat_dist[dist<25000] = 9.
cat_dist[dist<20000] = 7.
cat_dist[dist<15000] = 5.
cat_dist[dist<10000] = 3.
cat_dist[dist<7000] = 2.
cat_dist[dist<5000] = 1.
cat_dist[dist<1000] = 0.

In [ ]:
# On the mesh
totcost = cost*20.+cat_dist

# On the uniform grid
distances, indices = tin_tree.query(xyi, k=3)

onIDs = np.where(distances[:, 0] == 0)[0]
inIDs = np.where(distances[:, 0] > 0)[0]

weights = np.ones(distances.shape)
weights[inIDs,:] = 1.0 / distances[inIDs,:] ** 2

denum = 1.0 / np.sum(weights, axis=1)

ncosti = np.sum(weights * totcost[indices], axis=1) * denum
if len(onIDs) > 0:
    ncosti[onIDs] = totcost[indices[onIDs, 0]]
ncosti = np.reshape(ncosti, badland_topo.x.shape)

# Smooth cost
# ncosti = gaussian_filter(ncosti, sigma=0.1)
np.place(ncosti, badland_topo.z < sealevel, 9999)

icost = ncosti.astype(dtype=int)

In [ ]:
# vtkfile2 = 'totcost'+str(step)+'.vtk'
# vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, 
#                                    point_data={"Z":elev, "ED":ED, 
#                                                "LEC":cost, 'FA':logFA, 'riverdist':cat_dist,
#                                                "cost":totcost
#                                               })
# meshio.write(vtkfile2, vis_mesh)

Sumatra, Malay P., Borneo, Java

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
ax.imshow(np.flipud(icost), vmin=0, vmax=50, cmap=cm.RdBu_r) 
ax.imshow(np.flipud(badland_topo.z-sealevel), vmin=-500, vmax=500, cmap=cm.RdBu) 

plt.scatter(220,80, s=80, facecolors='w', edgecolors='k') # Gulf of Thailand
plt.scatter(460,400, s=80, facecolors='tab:green', edgecolors='k') # Borneo
plt.scatter(180,340, s=80, facecolors='tab:blue', edgecolors='k') # Sumatra
plt.scatter(420,580, s=80, facecolors='tab:red', edgecolors='k') # Java

# print(badland_topo.z[720-80,220]-sealevel)
# print(icost[720-80,220],icost.shape)
plt.show()

# Defining the migration paths

First we define the starting and destination points:

In [ ]:
bioPts = np.zeros((4,2),dtype=int)
bioPts[0,0] = 220
bioPts[0,1] = 80
bioPts[1,0] = 460
bioPts[1,1] = 400
bioPts[2,0] = 180
bioPts[2,1] = 340
bioPts[3,0] = 420
bioPts[3,1] = 580

rcPts = []
rcDst = []

rcPts.append(np.flipud(bioPts[0,:])*[-1,1]+[icost.shape[0],0])
rcDst.append(np.flipud(bioPts[1,:])*[-1,1]+[icost.shape[0],0])
rcPts.append(np.flipud(bioPts[0,:])*[-1,1]+[icost.shape[0],0])
rcDst.append(np.flipud(bioPts[2,:])*[-1,1]+[icost.shape[0],0])
rcPts.append(np.flipud(bioPts[0,:])*[-1,1]+[icost.shape[0],0])
rcDst.append(np.flipud(bioPts[3,:])*[-1,1]+[icost.shape[0],0])
rcPts.append(np.flipud(bioPts[1,:])*[-1,1]+[icost.shape[0],0])
rcDst.append(np.flipud(bioPts[2,:])*[-1,1]+[icost.shape[0],0])
rcPts.append(np.flipud(bioPts[1,:])*[-1,1]+[icost.shape[0],0])
rcDst.append(np.flipud(bioPts[3,:])*[-1,1]+[icost.shape[0],0])
rcPts.append(np.flipud(bioPts[2,:])*[-1,1]+[icost.shape[0],0])
rcDst.append(np.flipud(bioPts[3,:])*[-1,1]+[icost.shape[0],0])

starts = [[rcPts[0].tolist()],[rcPts[1].tolist()],[rcPts[2].tolist()],
          [rcPts[3].tolist()],[rcPts[4].tolist()],[rcPts[5].tolist()]]
ends = [[rcDst[0].tolist()],[rcDst[1].tolist()],[rcDst[2].tolist()],
          [rcDst[3].tolist()],[rcDst[4].tolist()],[rcDst[5].tolist()]]

## Least cost path

First we compute the distance cost from one point to every other points... (not useful here)

In [ ]:
m = MCP(icost,fully_connected=True)
for s in range(6):

    costTrace = 'traces/costTrace'+str(step)+'_'+str(s)+'.csv'
    
    # cost_array, tracebacks_array = m.find_costs(starts1, ends1)
    cost_array, tracebacks_array = m.find_costs(starts[s], ends[s])

    # Transpose `ends` so can be used to index in NumPy
    ends_idx = tuple(np.asarray(ends).T.tolist())
    costs = cost_array[ends_idx]

    # Compute exact minimum cost path to each endpoint
    tracebacks = [m.traceback(end) for end in ends[s]]
    trace = np.asarray(tracebacks[0])
    path1 = np.zeros((len(trace),4))
    
    print(len(trace))
    for k in range(len(trace)):
        path1[k,0] = badland_topo.x[trace[k][0],trace[k][1]]
        path1[k,1] = badland_topo.y[trace[k][0],trace[k][1]]
        path1[k,2] = badland_topo.z[trace[k][0],trace[k][1]]
        path1[k,3] = cost_array[trace[k][0],trace[k][1]]

    df = pd.DataFrame({'X':path1[:,0],'Y':path1[:,1],'Z':path1[:,2], 'tcost': path1[:,3]})

    df.to_csv(costTrace,columns=['X', 'Y', 'Z', 'tcost'], sep=',', index=False)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
ax.imshow(np.flipud(icost),vmin=0, vmax=50, cmap=cm.RdBu_r) 

trace = np.asarray(tracebacks[0])
for k in range(len(trace)):
    ax.scatter(trace[k][1], icost.shape[0]-trace[k][0], s=10, c='orange', marker='o', zorder=3, edgecolor='yellow')
    
plt.show()

## Circuitscape current map

First we reduce the map size...

In [ ]:
icost[icost<1] = 1
loc_cost = np.flipud(icost)
f = StringIO()
np.savetxt(f, loc_cost, fmt='%.3f')
f.seek(0)
fs = f.read().replace('.000', '', -1)
f.close()

In [ ]:
f = open('traces/cost'+str(step)+'.asc', 'w')
f.write("ncols " + str(loc_cost.shape[1]) + "\n")
f.write("nrows " + str(loc_cost.shape[0]) + "\n")
f.write("xllcorner " + str(0) + "\n")
f.write("yllcorner " + str(0) + "\n")
f.write("cellsize " + str(5000) + "\n")
f.write("NODATA_value " + str(9999) + "\n")
f.write(fs)
f.close()

In [ ]:
locs = np.zeros((4,2),dtype=int)
for k in range(4):
    locs[k,0] = (bioPts[k,0])*5000
    locs[k,1] = (icost.shape[0]-bioPts[k,1])*5000

vv = (locs/5000).astype(int)
print('cost for each points (needs to be above 9999)',icost[vv[:,1],vv[:,0]])

df = pd.DataFrame({'X':locs[:,0],'Y':locs[:,1]})
df.to_csv('traces/pts'+str(step)+'.txt',columns=['X', 'Y'], sep=' ', index=True, header=0)

You can use circuitscape to do a pairwise analysis and get the current map... Here we load the resulting map:

In [ ]:
curmap  = np.flipud(np.loadtxt('traces/'+str(step)+'_cum_curmap.asc', skiprows=6))
curmap1  = np.flipud(np.loadtxt('traces/'+str(step)+'_curmap_0_1.asc', skiprows=6))
curmap2  = np.flipud(np.loadtxt('traces/'+str(step)+'_curmap_0_2.asc', skiprows=6))
curmap3  = np.flipud(np.loadtxt('traces/'+str(step)+'_curmap_0_3.asc', skiprows=6))
curmap4  = np.flipud(np.loadtxt('traces/'+str(step)+'_curmap_1_2.asc', skiprows=6))
curmap5  = np.flipud(np.loadtxt('traces/'+str(step)+'_curmap_1_3.asc', skiprows=6))
curmap6  = np.flipud(np.loadtxt('traces/'+str(step)+'_curmap_2_3.asc', skiprows=6))

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.imshow(np.flipud(curmap), cmap=cm.Oranges, vmin=0, vmax=0.1)
plt.show()

# Write bio map as vtk file

In [ ]:
tree = cKDTree(xyi) 

In [ ]:
res = curmap.flatten()
res1 = curmap1.flatten()
res2 = curmap2.flatten()
res3 = curmap3.flatten()
res4 = curmap4.flatten()
res5 = curmap5.flatten()
res6 = curmap6.flatten()

# On the uniform grid
distances, indices = tree.query(XY, k=3)

onIDs = np.where(distances[:, 0] == 0)[0]
inIDs = np.where(distances[:, 0] > 0)[0]

weights = np.ones(distances.shape)
weights[inIDs,:] = 1.0 / distances[inIDs,:] ** 2

denum = 1.0 / np.sum(weights, axis=1)

resmap = np.sum(weights * res[indices], axis=1) * denum
resmap1 = np.sum(weights * res1[indices], axis=1) * denum
resmap2 = np.sum(weights * res2[indices], axis=1) * denum
resmap3 = np.sum(weights * res3[indices], axis=1) * denum
resmap4 = np.sum(weights * res4[indices], axis=1) * denum
resmap5 = np.sum(weights * res5[indices], axis=1) * denum
resmap6 = np.sum(weights * res6[indices], axis=1) * denum
if len(onIDs) > 0:
    resmap[onIDs] = res[indices[onIDs, 0]]
    resmap1[onIDs] = res1[indices[onIDs, 0]]
    resmap2[onIDs] = res2[indices[onIDs, 0]]
    resmap3[onIDs] = res3[indices[onIDs, 0]]
    resmap4[onIDs] = res4[indices[onIDs, 0]]
    resmap5[onIDs] = res5[indices[onIDs, 0]]
    resmap6[onIDs] = res6[indices[onIDs, 0]]

In [ ]:
vtkfile2 = 'vtk/bioMap'+str(step)+'.vtk'
vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, 
                       point_data={"Z":elev, "ED":ED, 
                       "LEC":cost, 'FA':logFA, 'riverdist':cat_dist,
                       "cost":totcost,"res_cum":resmap,
                       "res_Thai_Borneo":resmap1,"res_Thai_Sumatra":resmap2,
                       "res_Thai_Java":resmap3,"res_Borneo_Sumatra":resmap4,
                       "res_Borneo_Java":resmap5,"res_Sumatra_Java":resmap6,
                      })
meshio.write(vtkfile2, vis_mesh)